# Testing PUDL DB Initialization
Testing the initialization of the PUDL DB can be a little involved, since you need to set up an environment similar to that of the init_db function in pudl.py, and then focus on whichever small part of it you're changing -- adding a new ingest function, data source, etc. Running the high level tests takes too long to iterate quickly and fix things, because it does *everything*.  This notebook sets up that init_db environment, and makes it easier to just focus on the things you're changing.

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import sys
import os
sys.path.append(os.path.abspath(os.path.join('..','..')))
from pudl import pudl, ferc1, eia923, eia860, settings
from pudl import constants as pc
from pudl import models, models_ferc1, models_eia923, models_eia860
from pudl import clean_eia923, clean_ferc1, clean_pudl, clean_eia860
from pudl import analysis, datastore, outputs
import numpy as np
import pandas as pd
import sqlalchemy as sa
import matplotlib.pyplot as plt
import matplotlib as mpl
from IPython.display import HTML
pd.options.display.max_columns = 100

## Define the scope of the test
We don't need to test everything every time. Which years and data sources are we doing?

In [3]:
#ferc1_tables  = pc.ferc1_default_tables
#ferc1_years   = pc.ferc1_working_years
ferc1_tables  = []
ferc1_years   = [2015,]

#eia860_tables = pc.eia860_pudl_tables
#eia860_years  = pc.eia860_working_years
eia860_tables = ['generators_eia860',]
eia860_years  = pc.eia860_working_years

#eia923_tables = pc.eia923_pudl_tables
#eia923_years  = pc.eia923_working_years
eia923_tables = []
eia923_years  = [2015,]

## We need to have a (nominally) working FERC DB to pull from

In [5]:
ferc1_engine = ferc1.db_connect_ferc1(testing=True)
ferc1.init_db(ferc1_tables=ferc1_tables,
              refyear=max(ferc1_years),
              years=ferc1_years,
              def_db=True,
              verbose=True,
              testing=True)


Defining new FERC Form 1 DB based on 2015...
Clearing any existing FERC Form 1 database MetaData...
Ingesting FERC Form 1 Data from 2015...


## Initialize the PUDL DB

In [6]:
pudl_engine = pudl.db_connect_pudl(testing=True)
pudl.init_db(ferc1_tables  = ferc1_tables,
             ferc1_years   = ferc1_years,
             eia923_tables = eia923_tables,
             eia923_years  = eia923_years,
             eia860_tables = eia860_tables,
             eia860_years  = eia860_years,
             verbose=True, debug=False, testing=True,
             csvdir=os.path.join(settings.PUDL_DIR, 'results', 'csvdump'),
             keep_csv=True)

Ingesting static PUDL tables...
Sniffing EIA923/FERC1 glue tables...
Reading EIA 860 spreadsheet data for 2011.
Reading EIA 860 spreadsheet data for 2012.
Reading EIA 860 spreadsheet data for 2013.
Reading EIA 860 spreadsheet data for 2014.
Reading EIA 860 spreadsheet data for 2015.
Converting EIA 860 boiler_generator_assn to DataFrame...
Reading EIA 860 spreadsheet data for 2011.
Reading EIA 860 spreadsheet data for 2012.
Reading EIA 860 spreadsheet data for 2013.
Reading EIA 860 spreadsheet data for 2014.
Reading EIA 860 spreadsheet data for 2015.
Converting EIA 860 utility to DataFrame...
Reading EIA 860 spreadsheet data for 2011.
Reading EIA 860 spreadsheet data for 2012.
Reading EIA 860 spreadsheet data for 2013.
Reading EIA 860 spreadsheet data for 2014.
Reading EIA 860 spreadsheet data for 2015.
Converting EIA 860 plant to DataFrame...
Reading EIA 860 spreadsheet data for 2011.
Reading EIA 860 spreadsheet data for 2012.
Reading EIA 860 spreadsheet data for 2013.
Reading EIA 860 

ProgrammingError: syntax error at or near "current_date"
LINE 1: ...code,uprate_derate_during_year,winter_capacity_mw,current_da...
                                                             ^


In [9]:
eia860_dfs = pudl.create_dfs_eia860(files=pc.files_eia860, eia860_years=eia860_years, verbose=True)


Reading EIA 860 spreadsheet data for 2011.
Reading EIA 860 spreadsheet data for 2012.
Reading EIA 860 spreadsheet data for 2013.
Reading EIA 860 spreadsheet data for 2014.
Reading EIA 860 spreadsheet data for 2015.
Converting EIA 860 boiler_generator_assn to DataFrame...
Reading EIA 860 spreadsheet data for 2011.
Reading EIA 860 spreadsheet data for 2012.
Reading EIA 860 spreadsheet data for 2013.
Reading EIA 860 spreadsheet data for 2014.
Reading EIA 860 spreadsheet data for 2015.
Converting EIA 860 utility to DataFrame...
Reading EIA 860 spreadsheet data for 2011.
Reading EIA 860 spreadsheet data for 2012.
Reading EIA 860 spreadsheet data for 2013.
Reading EIA 860 spreadsheet data for 2014.
Reading EIA 860 spreadsheet data for 2015.
Converting EIA 860 plant to DataFrame...
Reading EIA 860 spreadsheet data for 2011.
Reading EIA 860 spreadsheet data for 2012.
Reading EIA 860 spreadsheet data for 2013.
Reading EIA 860 spreadsheet data for 2014.
Reading EIA 860 spreadsheet data for 2015.

In [10]:
gens_df = pudl.ingest_generators_eia860(pudl_engine, eia860_dfs)

In [15]:
gens_df.to_sql('generators_eia860', pudl_engine, if_exists='replace', chunksize=1000)

## Clean up after ourselves

In [ ]:
ferc1.drop_tables_ferc1(ferc1_engine)
pudl.drop_tables_pudl(pudl_engine)